# STAT413 Homework 5
Lauren Huang

## Problem 2
Instructions: Write a brief explanation of the code and show your results. You can explore the code by varying the design parameters.

In [ ]:
! pip install torch numpy transformers datasets tiktoken wandb tqdm

Note: you may need to restart the kernel to use updated packages.


In [8]:
! python data/shakespeare_char/prepare.py

length of dataset in characters: 1,115,394
all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65
train has 1,003,854 tokens
val has 111,540 tokens


I do not have a GPU, so I will be following this set of instructions.

In [9]:
! python train.py config/train_shakespeare_char.py --device=cpu --compile=False --eval_iters=20 --log_interval=1 --block_size=64 --batch_size=12 --n_layer=4 --n_head=4 --n_embd=128 --max_iters=2000 --lr_decay_iters=2000 --dropout=0.0

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of 

c:\Users\cupca\OneDrive\Documents\GitHub\nanoGPT\train.py:196: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))
c:\Users\cupca\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [10]:
! python sample.py --out_dir=out-shakespeare-char --device=cpu

Overriding: out_dir = out-shakespeare-char
Overriding: device = cpu
number of parameters: 0.80M
Loading meta from data\shakespeare_char\meta.pkl...

Theren:
For But I fair pleadion, no do all thy Egal
Than heaves aginss were brey: not.

Sevire:
Nother is befy well he the honoun him the in the
Our the care as so for honou fall the wippea,
Which perte our thing him.

TLEONENIUS:
Say, go lies sir.

GLood, thou dow it are a their be.

CLOLIBET:
Hell not stort:
Theing wo not with in blay to seet a onde thess
And but hould the it rie storde carition word wound,
Pecarnury muspe more he hath unter the the to Give shy demay's
The say enter then the a
---------------

Rechild but to boftick true a bust sough;
Not be see in hand then hourse for deater,
And wee his, are have as a dest do of more,
That here you, this to my my spead: I parother
And that it the applain at and thy shall the be are
mistech, in ampin ouce the brow our my all before
A our to sorre then soul plesting speake lie
the have w

c:\Users\cupca\OneDrive\Documents\GitHub\nanoGPT\sample.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=device)


In [11]:
! python data/openwebtext/prepare.py

c:\Users\cupca\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cupca\.cache\huggingface\hub\datasets--openwebtext. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Traceback (most recent call last):
  File "c:\Users\cupca\OneDrive\Documents\GitHub\nanoGPT\data\open

In [12]:
! torchrun --standalone --nproc_per_node=8 train.py config/train_gpt2.py

W1102 18:27:53.279000 14472 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W1102 18:27:53.441000 14472 site-packages\torch\distributed\run.py:793] 
W1102 18:27:53.441000 14472 site-packages\torch\distributed\run.py:793] *****************************************
W1102 18:27:53.441000 14472 site-packages\torch\distributed\run.py:793] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1102 18:27:53.441000 14472 site-packages\torch\distributed\run.py:793] *****************************************
Traceback (most recent call last):
  File "C:\Users\cupca\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\distributed\elastic\rendezvous\c10d_rendezvous_backend.py", line 159, in _create_tcp_store
    store = TCPStore(
            ^^^^

In [13]:
# Run on the first (master) node with example IP 123.456.123.456:
! torchrun --nproc_per_node=8 --nnodes=2 --node_rank=0 --master_addr=123.456.123.456 --master_port=1234 train.py
# Run on the worker node:
! torchrun --nproc_per_node=8 --nnodes=2 --node_rank=1 --master_addr=123.456.123.456 --master_port=1234 train.py

W1102 18:28:14.279000 16160 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W1102 18:28:14.313000 16160 site-packages\torch\distributed\run.py:793] 
W1102 18:28:14.313000 16160 site-packages\torch\distributed\run.py:793] *****************************************
W1102 18:28:14.313000 16160 site-packages\torch\distributed\run.py:793] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1102 18:28:14.313000 16160 site-packages\torch\distributed\run.py:793] *****************************************
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\cupca\AppData\Local\Programs\Python\Python311\Scripts\torchrun.exe\__main__.py", line 7, in <m